In [1]:
# 필요한 라이브러리 생성란

import json

# 데이터 처리 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 머신러닝 라이브러리
import scipy.optimize as opt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 데이터 불러오는 라이브러리
from datasets import *

# 에빙하우스의 망각 곡선을 이용한 회귀 분석

[에빙하우스의 망각곡선 내용 바로가기](https://kangpro-safety.tistory.com/43)

망각 곡선을 이용한 회귀분석을 진행하기 위해 다음과 같은 방식으로 진행하였다.

망각 곡선의 간략식은 다음과 같다.<br/>

$ R(t) = R_0e^{-kt} $
- $R(t)$ : 시간 t초 후 기억 유지율
- $R_0$ : 초기 기억수준
- $k$ : 망각 속도 계수
- $t$ : 시간

---
## 1. 시작하기 앞서

위의 방정식을 이용하기 전 데이터 EDA를 통해서 알 수 있는 부분은 다음과 같다.
- 각 유저 개인이 컨텐츠를 시작하고 완료하기 까지 걸린 시간
- 각 유저 개인이 레슨을 시작하고 완료하기 까지 걸린 시간

그리고 데이터EDA를 했음에도 알 수 없는 내용은 다음과 같았다.
- 교육 플랫폼이 어떤 교육을 기반으로 운영하고 있는지

---
## 2. 망각 속도 계수 함수 정의

에빙하우스의 망각 곡선 방정식에서 구할 수 있는 상수는 **망각 속도 계수** $k$로 지정하였다.<br/>
다른 상수를 구할 수 없는 이유는 다음과 같다.
- 초기 기억수준($R_0$)은 유저의 학습 후 테스트(시험)에 대한 데이터가 있어야 진행할 수 있는데 데이터가 존재하지 않아 사용할 수 없다.
- 시간($t$)은 우리가 사용하고자 하는 총 걸린 시간을 이야기 하는 것이 아닌, 시간이 지남에 따른 연속적인 수치를 사용하므로, 값이 하나인 것에는 대응하기 어려울 수 있다고 생각한다.

In [2]:
# 망각 곡선 모델 정의
def forgetting_curve(I, k0, alpha):
    return k0 * np.exp(-alpha * I)

망각 속도 계수를 구하는 방정식(모델)은 다음과 같이 정의할 수 있다.

$k = k_0e^{-\alpha I}$

1. $k_0$ : **기본 망각 계수**
    - 앞서 이야기한 교육 플랫폼이 어떤 교육을 기반으로 운영되고 있는지 데이터로 확인할 수 없었다.
    - 각 컨텐츠 소비 시간 패턴인 학습 집중도 비율값인 $I$를 통해 해당 플랫폼이 어떤 교육을 기반으로 하는지 파악할 수 있다.

In [3]:
# 학습 집중도 비율
def LCR(T_L,  T_C):
    return T_L / T_C

2. $I = \frac {T_L} {T_C}$ : 학습 집중도 비율
    - $I \approx 1.0$ (거의 모든 시간을 레슨에 소비) → 집중 학습
    - $I < 1.0$ (일부만 레슨 학습) → 실습 위주 학습 가능성
        - $I \approx 0.5$ (절반만 레슨 학습)
        - $I \approx 0.2$ (짧은 시간만 레슨 학습)
    
    이 값을 각 유저 개인마다 계산하여 다음과 같은 절차를 거친다.
    1. K-means 으로 군집화를 통한 Clustering 적용
    2. 클러스터를 분석하여 높은 $I$ 그룹이 실제 암기/개념 학습인지 검증<br/>
        → 검증은 어떻게?

        - 복습 빈도
            - 암기/개념 이해 학습의 경우 짧은 간격으로 반복 학습하고, 같은 콘텐츠를 여러 번 조회
            - 실습 학습의 경우 처음 학습 후 재방문이 적다. (즉 위의 말과 반대된다.)

        - 콘텐츠 재방문 시간 간격
            - Ebbinghaus 망각 곡선 이론에 따르면, 복습 패턴은 학습 유형과 관련이 있다.
            - 복습이 빠르게 이루어지면 암기 가능성, 복습 없이 실습이 많다면 실습 가능성

위의 내용을 통해 $k_0$값을 실제 데이터와 잘 맞게 하기 위해 다음과 같은 함수식을 사용하였다.

In [4]:
# 기본적인 망각 계수
def k0_exponential(I):
    return 0.3 - 0.2 * np.exp(-3 * I)

3. $\alpha$ : 학습 집중도에 따른 망각 속도 감소 계수<br/>
    위의 학습 집중도 비율을 통해 Clustering한 군집들에 대해서 다음과 같은 값을 적용
    - $I = 1.0$ → 3.0
    - $I = 0$ → 1.5

여기서 $\alpha$값을 실제 학습 과정에서 $\alpha$ 변화를 현실적인 패턴을  반영하기 위해 다음과 같은 함수식을 작성하였다.

In [5]:
# 학습 집중도에 따른 망각 속도 감소 계수
def alpha_exponential(number):
    return 3.0 - 1.5 * np.exp(-3 * number)

---
## 3. 데이터 전처리 및 데이터 준비

In [15]:
# 인코딩된 데이터 불러오기 
df = pd.read_csv('./lwc_encoder/merged_encoded_data.csv')

# 디코딩을 위한 데이터 불러오기
with open('./lwc_encoder/encoding_map.json', 'r') as f:
    json_data = json.load(f)

C:\Users\lento\AppData\Local\Temp\ipykernel_14660\640150351.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./lwc_encoder/merged_encoded_data.csv')


In [16]:
# 회귀분석에 필요한 컬럼을 불러오기 위한 컬럼 이름 확인
df.columns

Index(['os_name', 'language', 'client_event_time', 'country', 'device_family',
       'user_id', 'event_type', 'city', 'content.id', 'button.name',
       'button_name', 'lesson.id', 'question.id', 'type',
       'coupon.discount_amount', 'pg.type', 'paid_amount', 'plan.price',
       'is_free_trial', 'is_trial', 'content.difficulty', 'trial.type',
       'plan.type'],
      dtype='object')

In [17]:
# 필요한 컬럼만 따로 불러오기
df = df[['user_id', 'event_type', 'content.id', 'content.difficulty', 'lesson.id', 'client_event_time']]

In [18]:
# 전체 데이터 크기 확인
df.shape[0]

33360653

In [19]:
# 결측치 확인
df.isnull().sum()

user_id                 596829
event_type                   0
content.id             3120112
content.difficulty    33235736
lesson.id              5546741
client_event_time            0
dtype: int64

In [20]:
# content.difficulty는 각 content.id에 맞는 값을 할당

# 결측값이 있는 행만 업데이트
mask = df['content.difficulty'].isna()
df.loc[mask, 'content.difficulty'] = df.groupby('content.id')['content.difficulty'].transform(lambda x: x.ffill().bfill())[mask]

In [21]:
# 결측치 확인
df.isnull().sum()

user_id                596829
event_type                  0
content.id            3120112
content.difficulty    5938440
lesson.id             5546741
client_event_time           0
dtype: int64

결측치 추측은 다음과 같다.
- user_id : 아이디가 없는 회원 (비회원)
- content.id : content.id가 존재하지 않음
- content.difficulty : content.id 내 난이도 내역이 존재하지 않음
- lesson.id : lesson.id가 존재하지 않음

In [ ]:
# 결측치가 존재하는 부분을 dropna 처리
df = df.dropna(subset=['user_id'])

In [24]:
# 결측치 확인
df.isnull().sum()

user_id               0
event_type            0
content.id            0
content.difficulty    0
lesson.id             0
client_event_time     0
dtype: int64

In [25]:
# 전체 데이터 크기 확인
df.shape[0]

25681491

In [28]:
# 난이도 데이터 분포 확인
# 난이도를 제외한 데이터는 명목형 데이터에 속함
# user_id, content.id, lesson.id, event_type - 명목형
# content.difficulty - 순서형
# 순서형의 분포가 어떻게 되어있는지 확인하기 위함
df[['content.difficulty']].describe()

,content.difficulty
count,2.568149e+07
mean,1.493191e+00
std,1.377789e+00
min,0.000000e+00
25%,0.000000e+00
50%,1.000000e+00
75%,3.000000e+00
max,3.000000e+00


In [30]:
# 시간 계산을 위한 데이터 타입 변경
df['client_event_time'] = pd.to_datetime(df['client_event_time'])

In [29]:
# event_type 중 다음 항목이 없으면 삭제
# start.content, enter.lesson_page, complete.lesson, end.content

event_list = [json_data['event_type']['start.content'],
              json_data['event_type']['end.content'],
              json_data['event_type']['enter.lesson_page'],
              json_data['event_type']['complete.lesson']]

df = df[df['event_type'].isin(event_list)]

In [31]:
df

,user_id,event_type,content.id,content.difficulty,lesson.id,client_event_time
1663904,31951.0,4,96.0,1.0,3533.0,2023-04-05 21:55:51.836
1663905,14554.0,4,40.0,0.0,2095.0,2023-04-05 21:19:01.243
1663906,14554.0,4,40.0,0.0,3509.0,2023-04-05 21:20:27.939
1663907,14554.0,4,40.0,0.0,4069.0,2023-04-05 21:23:01.534
1663908,14554.0,4,40.0,0.0,3438.0,2023-04-05 21:28:34.965
...,...,...,...,...,...,...
30292936,173526.0,9,158.0,0.0,1822.0,2023-10-10 05:18:14.053
30292937,173526.0,9,158.0,0.0,2845.0,2023-10-10 05:18:58.648
30292938,127564.0,9,200.0,1.0,2073.0,2023-10-10 05:17:45.578
30292939,127564.0,9,200.0,1.0,3778.0,2023-10-10 05:18:05.629
